# A pytorch ML adapter demo

 1. [Pytorch](#pytorch) model example.
 2. Create [Adapter](#adapter).
 3. How to create a [Webscript](#webscript).
 4. How to create a [Plug](#plug).

#### Preamble
Demo of a relatively small pytorch model.
This notebook uses the [ml_adapter_torch](../../env/ml_adapter_torch) dependencies.
To start: 
```
bin/jupyter_notebook env/ml_adapter_torch ml_adapter/torch_autoencoder
```

In [1]:
!python --version

Python 3.11.9


In [2]:
# the sdk profile used to connect
PROFILE='_default_'
LOG_LEVEL='INFO'
MODEL_NAME='autoencoderV1'
MODEL_VERSION='1.0.0'

In [3]:
# setup INFO logging to see http requests made.
import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=LOG_LEVEL,
    datefmt='%Y-%m-%d %H:%M:%S'
)

## 1. Pytorch example <a id="pytorch"></a>

A simple auto-encoder pytorch model.

In [4]:
from autoencoder import AutoEncoder

In [5]:
# we saved our model class in a `autoencoder.py` file
from IPython.display import Code, Markdown
display(Code(filename='autoencoder.py'))
from autoencoder import AutoEncoder

import torch

class AutoEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        """
        Create a simple AutoEncoder
        """
        # Use an AutoEncoder and try to reconstruct both signals (the 20 samples back)
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 5),
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(5, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 20),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [6]:
# some pretrained weights
weights_path = 'AutoEncoderWeights.pth'

In [7]:
import torch
model = AutoEncoder()
model.load_state_dict(torch.load(weights_path))
model.eval()

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=5, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
  )
)

In [8]:
x_data = torch.randn(20, dtype=torch.float32)
x_data

tensor([ 0.1732,  0.0324,  0.0194,  0.8020, -1.3497,  0.8811, -1.9011, -0.4583,
        -1.8547,  0.5366,  1.2801, -0.4675, -1.5484,  0.8876,  0.1250, -0.4512,
         2.4818, -1.4263,  1.6530,  0.1223])

In [9]:
preds = model(x_data)
preds

tensor([-0.0840, -0.1075, -0.1180, -0.0856, -0.1301, -0.1541, -0.1372, -0.1259,
        -0.1530, -0.0339, -0.1280, -0.0944, -0.0869, -0.1309, -0.1258, -0.1441,
        -0.0714, -0.1667, -0.1455, -0.1114], grad_fn=<ViewBackward0>)

## 2. The adapter <a id="adapter"></a>
The `V1TorchAdapter` from the `ml_adapter.torch` module wraps our model in a script that can be used as a waylay webscript or plug.

In [10]:
# choose a local archive location
ARCHIVE_LOC = 'autoencoder-pytorch'
# make sure its empty
!rm -fr autoencoder-pytorch

In [11]:
from ml_adapter.torch import V1TorchAdapter
# create an ML adapter to wrap our model
# by using a `weights.pt` postfix we are storing only the weights when serializing the model
MODEL_PATH='model-weights.pt'
adapter = V1TorchAdapter(model=model, model_path='model-weights.pt', location=ARCHIVE_LOC)

In [12]:
# the adapter exposes your model with a REST-compatible interface
result = await adapter.call({"instances": [ x_data.tolist() ]}) 
result

{'predictions': [[-0.0839521661400795,
   -0.10749989002943039,
   -0.11796973645687103,
   -0.08562793582677841,
   -0.13007289171218872,
   -0.1541411578655243,
   -0.1371534764766693,
   -0.12586581707000732,
   -0.15304678678512573,
   -0.03389810025691986,
   -0.12804251909255981,
   -0.0943613275885582,
   -0.08690498769283295,
   -0.13089188933372498,
   -0.12576353549957275,
   -0.14411213994026184,
   -0.07141607999801636,
   -0.16669408977031708,
   -0.1455150693655014,
   -0.11138710379600525]]}

In [13]:
# because we store only weights, the adapter archive needs to now about autoencode model class:
await adapter.add_script('autoencoder.py')

autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>

In [14]:
await adapter.save()
# have a look at ARCHIVE_LOC to see the stored assets
list(a.path for a in adapter.assets)

['openapi.json',
 'webscript.json',
 'requirements.txt',
 'main.py',
 'model-weights.pt',
 'autoencoder.py']

## 3. Creating the webscript <a id="webscript"></a>
Tell the adapter to configure itself as a webscript: this generates a number of _assets_ that will be uploaded and define the webscript behaviour
* a `webscript.json` _manifest_ file that defines the name, version, _runtime_, deploy settings, metadata ...
* a `requirements.txt` package dependencies file
* a `main.py` webscript script
* additional scripts we added above, like the `autoencoder.py` and the `model-weights.pt`

In [15]:
## configure any 'memory' or 'cpu' deploy settings 
deploy_overrides = {'limits' : { 'memory': '2G' }, 'requests' : { 'memory' : '1G' }}
## configure the webscript to use our model
adapter = adapter.as_webscript({
    'name': MODEL_NAME, 
    'description':'pytorch autoencoder for caats', 
    'deploy' : deploy_overrides                                                                                          
})

In [16]:
await adapter.save()
list(a.path for a in adapter.assets)

['openapi.json',
 'webscript.json',
 'requirements.txt',
 'main.py',
 'model-weights.pt',
 'autoencoder.py']

In [17]:
# lets have a look at the generated python webscript:
display(Code(filename=f'{ARCHIVE_LOC}/main.py'))

# You could adapt this script to have specific error handling or handling of request/response

# ml_adapter.torch.adapter.V1TorchAdapter model adapter
import os
from starlette.requests import Request
from starlette.responses import JSONResponse
from starlette.exceptions import HTTPException
from ml_adapter.torch.adapter import V1TorchAdapter

MODEL_PATH = os.environ.get('MODEL_PATH', 'model-weights.pt')
MODEL_CLASS = os.environ.get('MODEL_CLASS', 'autoencoder.AutoEncoder')

# Initialize the model adapter.
# Provide a `model` argument if you want to create/load the model yourself.
adapter = V1TorchAdapter(
    model_path=MODEL_PATH, model_class=MODEL_CLASS
)

# Webscript handler
async def execute(request: Request):
    if request.method == 'GET':
        return JSONResponse(adapter.openapi)
    if request.method != 'POST':
        raise HTTPException(
            status_code=405,
            detail='This webscript only accepts `POST` calls.',
        )
    # use request body as input
    request_json = await request.json()
    # call the model adapter using the V1
    response_json = await adapter.call(request_json)
    return JSONResponse(response_json)

In [18]:
# once stored, the adapter can be restored later with
adapter = await V1TorchAdapter(model_path='model-weights.pt', model_class=AutoEncoder, location=ARCHIVE_LOC).load()

In [19]:
list(adapter.assets)

[openapi.json <ml_adapter.base.assets.openapi.OpenApiAsset>,
 webscript.json <ml_adapter.base.assets.manifest.WebscriptManifestAsset>,
 requirements.txt <ml_adapter.base.assets.python.PythonRequirementsAsset>,
 main.py <ml_adapter.base.assets.python.PythonScriptAsset>,
 model-weights.pt <ml_adapter.torch.adapter.TorchModelWeightsAsset>,
 autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>]

### Uploading the webscript using the SDK
To upload these assets and create a webscript, we need to call the [create webcript](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Webscripts/operation/create_webscripts) REST api. 

The code belows uses the `ml_tool` plugin to handle this.
Alternatively you could call `await adapter.save_archive()`
which creates an `autoencoder-pytorch.tar.gz` archive that you can upload as webscript with `curl` or using the waylay console.

In [20]:
from waylay.sdk import WaylayClient

In [24]:
# check the SDK client
client = WaylayClient.from_profile('staging')

In [25]:
ref = await client.ml_tool.create_webscript(adapter)
ref

2024-06-11 11:47:25 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/accounts/v1/tokens?grant_type=client_credentials "HTTP/1.1 200 OK"
2024-06-11 11:47:26 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/registry/v2/webscripts/?draft=false&comment=&async=true "HTTP/1.1 202 Accepted"


{'message': 'Building and deploying webscript autoencoder-pytorch-v1@0.0.1',
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$BO6yELncbGqLydPhWzvFE&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/verify/740799ef-d515-4704-8718-903851c9899e$BO6yELncbGqLydPhWzvFE'}},
 'entity': {'createdBy': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
  'createdAt': '2024-06-11T09:47:26.176Z',
  'updatedBy': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
  'updatedAt': '2024-06-11T09:47:26.199Z',
  'updates': [{'operation': 'create',
    'at': '2024-06-11T09:47:26.199Z',
    'by': 'users/edb8841f-122e-4f7d-a412-397764bc9996',
    'comment': '',
    'jobs': ['740799ef-d515-4704-8718-903851c9899e$BO6yELncbGqLydPhWzvFE',
     '740799ef-d515-4704-8718-903851c9899e$1zoyGfeHuap8L6MdI9hlV',
     '740799ef-d515-4704-8718-903851c9899e$VQdbRM4WJxFgmjvIp4TrC']}],
  'status': 'pending',
  'runt

In [26]:
# wait until the build, deploy and verify jobs for the webscript have finished
# NOTE: this might take quite a few minutes, as the dependencies for torch webscripts are quite big and the images not yet optimised
ref = await client.ml_tool.wait_until_ready(ref)

2024-06-11 11:47:36 INFO     Waiting for autoencoder-pytorch-v1@0.0.1 to be ready:
2024-06-11 11:47:36 INFO     listening on https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$BO6yELncbGqLydPhWzvFE&children=true
2024-06-11 11:47:37 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$BO6yELncbGqLydPhWzvFE&children=true "HTTP/1.1 200 OK"
2024-06-11 11:47:37 INFO     autoencoder-pytorch-v1@0.0.1 deploy: waiting-children
2024-06-11 11:47:52 INFO     keep-alive: {}
2024-06-11 11:48:22 INFO     keep-alive: {}
2024-06-11 11:48:51 INFO     keep-alive: {}
2024-06-11 11:49:22 INFO     keep-alive: {}
2024-06-11 11:49:52 INFO     keep-alive: {}
2024-06-11 11:50:22 INFO     keep-alive: {}
2024-06-11 11:50:52 INFO     keep-alive: {}
2024-06-11 11:51:21 INFO     keep-alive: {}
2024-06-11 11:51:52 INFO     keep-alive: {}
2024-06-11 11:52:22 INFO     keep-alive: {}
202

In [27]:
# test the webscript invocation
await client.ml_tool.test_webscript(ref, x_data.tolist())

2024-06-11 12:03:12 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/functions/v1/740799ef-d515-4704-8718-903851c9899e/autoencoder-pytorch-v1 "HTTP/1.1 200 OK"


[-0.0839521661400795,
 -0.10749989002943039,
 -0.11796973645687103,
 -0.08562793582677841,
 -0.13007290661334991,
 -0.1541411578655243,
 -0.1371534764766693,
 -0.12586581707000732,
 -0.15304680168628693,
 -0.033898092806339264,
 -0.12804250419139862,
 -0.0943613275885582,
 -0.08690498769283295,
 -0.13089188933372498,
 -0.12576353549957275,
 -0.14411213994026184,
 -0.07141607999801636,
 -0.16669408977031708,
 -0.1455150693655014,
 -0.11138710379600525]

In [28]:
# remove the webscript
await client.ml_tool.remove(ref)

2024-06-11 12:57:21 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/webscripts/autoencoder-pytorch-v1/versions/0.0.1?force=true "HTTP/1.1 202 Accepted"


{'message': 'Deleting webscript autoencoder-pytorch-v1@0.0.1',
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=undeploy&id=740799ef-d515-4704-8718-903851c9899e$EPOirtvaoVc1LYUm-Lj5U&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/undeploy/740799ef-d515-4704-8718-903851c9899e$EPOirtvaoVc1LYUm-Lj5U'}},
 'versions': ['0.0.1']}

## 4. Creating the plug <a id="plug"></a>
Tell the adapter to configure itself as a plug: this generates a number of _assets_ that will be uploaded and define the plug behaviour
* a `plug.json` _manifest_ file that defines the name, version, _runtime_, deploy settings, metadata ...
* a `requirements.txt` package dependencies file
* a `main.py` webscript script
* additional scripts we added above, like the `autoencoder.py` and the `model-weights.pt`

#### Intiliaze adapter for plug deployment (see how to create [adapter](#adapter)).

In [ ]:
# choose a local archive location
ARCHIVE_LOC = 'autoencoder-pytorch'
# make sure its empty
!rm -fr autoencoder-pytorch

In [ ]:
from ml_adapter.torch import V1TorchAdapter
# create an ML adapter to wrap our model
# by using a `weights.pt` postfix we are storing only the weights when serializing the model
MODEL_PATH='model-weights.pt'
adapter = V1TorchAdapter(model=model, model_path='model-weights.pt', location=ARCHIVE_LOC)

In [ ]:
# the adapter exposes your model with a REST-compatible interface
result = await adapter.call({"instances": [ x_data.tolist() ]}) 
result

In [ ]:
## configure any 'memory' or 'cpu' deploy settings 
deploy_overrides = {'limits' : { 'memory': '2G' }, 'requests' : { 'memory' : '1G' }}
## configure the webscript to use our model
adapter = adapter.as_plug({
    'name': MODEL_NAME, 
    'description':'pytorch autoencoder for caats', 
    'deploy' : deploy_overrides                                                                                          
})

In [ ]:
await adapter.save()
list(a.path for a in adapter.assets)

In [ ]:
# lets have a look at the generated python plug:
display(Code(filename=f'{ARCHIVE_LOC}/main.py'))

# You could adapt this script to have specific error handling or handling of request/response

In [ ]:
list(a.path for a in adapter.assets)

### Uploading the plug using the SDK
To upload these assets and create a plug, we need to call the [create plug](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Plugs/operation/create_plugs) REST api. 

The code belows uses the `ml_tool` plugin to handle this.
Alternatively you could call `await adapter.save_archive()`
which creates an `autoencoder-pytorch.tar.gz` archive that you can upload as a plug with `curl` or using the waylay console.

In [2]:
from waylay.sdk import WaylayClient

In [ ]:
# check the SDK client
client = WaylayClient.from_profile('staging')

In [ ]:
ref = await client.ml_tool.create_plug(adapter)
ref

In [ ]:
# wait until the build, deploy and verify jobs for the webscript have finished
# NOTE: this might take quite a few minutes, as the dependencies for torch webscripts are quite big and the images not yet optimised
ref = await client.ml_tool.wait_until_ready(ref)

In [ ]:
# test the plug invocation
await client.ml_tool.test_plug(ref, x_data.tolist())

In [ ]:
# remove the plug
await client.ml_tool.remove(ref)

#### About `ml_tool`
The `client.ml_tool` methods are essentialy wrappers around the methods of the [registry](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Webscripts) service. Alternatively you can use the [`client.registry.webscript`](https://github.com/waylayio/waylay-sdk-registry-py) methods directly.